## Scraping [1/2]

In [ ]:
import requests
from bs4 import BeautifulSoup

# from classes import Episode
# episode = Episode()


def strToInt(str):
  try:
    return int(str)
  except:
    return 0


def elementToObject(element, date):
  episodeNameElements = element.findAll('a')
  seasonAndEpisode = episodeNameElements[1].text.split('.')
  channelElement = element.find_previous_sibling('img')
  contryElements = channelElement.find_previous_sibling('img')

  object = {
    'name': episodeNameElements[0].text,
    'episode': strToInt(seasonAndEpisode[1]),
    'season': strToInt(seasonAndEpisode[0]),
    'date': date,
    'country': contryElements.get('alt'),
    'channel': channelElement.get('alt'),
    'url': episodeNameElements[0].get('href')
  }
  return object


url = 'https://www.spin-off.fr/calendrier_des_series.html'
response = requests.get(url)
content = response.content
page = BeautifulSoup(content, 'html')
currentMonth = page.find_all('td', class_=['floatleftmobile td_jour', 'floatleftmobile td_jour td_jourcourant'])


episodes = []
for day in currentMonth:
  dateElement = day.find('div', class_=['div_jour', 'div_jourcourant fond_degrade_v4'])
  if dateElement:
    date = dateElement.get('id').split('_')[1]
    episodesElements = day.find_all('span', class_=['calendrier_episodes'])
    for episodeElement in episodesElements:
      episodes.append(elementToObject(episodeElement, date))

print(episodes)

## Fichiers

In [ ]:
def objectToStringforCsv(episode):
  return f'{episode['name']},{episode['episode']},{episode['season']},{episode['date']},{episode['country']},{episode['channel']},{episode['url']}\n'

strEpisodes = [
  objectToStringforCsv(episode)
  for episode in episodes
]

with open('./data/files/episodes.csv', mode='w+', encoding='utf-8') as file:
  file.write(''.join(strEpisodes))

In [ ]:
def lineToTuples(line):
  values = line.split(',')
  print(values)

# la method "arrayToEpisode" permet de récupérer les données depuis le fichier CSV plutôt que de refaire le scrapping du site.
def arrayToEpisode(values):
  return {
    'name': values[0],
    'episode': int(values[1]),
    'season': int(values[2]),
    'date': values[3],
    'country': values[4],
    'channel': values[5],
    'url': values[6]
  }

typeTuples = []
episodes = []
with open('./data/files/episodes.csv', 'r', encoding='utf-8') as file:
  for line in file:
    types = []
    values = line.strip().split(',')
    episodes.append(arrayToEpisode(values))
    for value in values:
      try:
        value = int(value)
      except:
        value = value
      types.append(type(value).__name__)
    typeTuples.append(tuple(types))

print(typeTuples[0])

## SQL [1/2]

In [11]:
import sqlite3

db = sqlite3.connect('./data/databases/database.db')
cursor = db.cursor()

tuples = [
  tuple(episode.values())
  for episode in episodes
]

req = 'insert into Episode (name, episode, season, date, country, channel, url) values (?, ?, ?, ?, ?, ?, ?)'
cursor.executemany(req, tuples) # /!\ Ne pas relancer cette partie du code si les données sont déjà présente dans la base
db.commit()
db.close()

## Algorithmie [1/2]

In [ ]:
import sqlite3

def countBy(key):
  db = sqlite3.connect('./data/databases/database.db')
  cursor = db.cursor()

  req = f'''
    select {key}, count(*) as length
    from Episode
    group by {key}
    order by length desc
    limit 3
  '''

  res = cursor.execute(req).fetchall()
  db.commit()
  db.close()

  return res



print(countBy('channel'))
print(countBy('country'))

In [ ]:
import sqlite3

db = sqlite3.connect('./data/databases/database.db')
cursor = db.cursor()

req = '''
  select name
  from Episode
  group by name
'''

res = cursor.execute(req).fetchall()
db.commit()
db.close()

counter = {}

for nameInArray in res:
  name = nameInArray[0]
  for word in name.split(' '):
    word = word.lower()
    try:
      counter[word] += 1
    except:
      counter[word] = 1

sortedCounter = dict(sorted(counter.items(), key=lambda item: item[1], reverse=True))

print(sortedCounter)

## Scraping [2/2]

In [ ]:
import time
import requests
from bs4 import BeautifulSoup

# for episode in episodes:
for index in range(10):
  episode = episodes[index]
  episodeName = episode
  url = f'https://www.spin-off.fr/{episode['url']}'

response = requests.get(url)
content = response.content
page = BeautifulSoup(content, 'html')
currentMonth = page.find_all('table', id_='series_liste_saisons')
print(currentMonth)

## SQL [2/2]

## Algorithmie [2/2]

## Orchestration